# Upload data to a S3 bucket

See example here:
http://datasciencedirectory.com/how-to-connect-to-aws-s3-buckets-with-python/

In [2]:
from pathlib import Path
import os
from configparser import ConfigParser
import boto3

In [3]:
repodir = Path("../../") 
inputdir = repodir / "data" / "output-report"
os.listdir(inputdir)

['fireveg-db-references.xlsx',
 'fireveg-field-report-model.xlsx',
 'fireveg-trait-records-curation.xlsx',
 'fireveg-trait-records-model.xlsx',
 'fireveg-trait-report-model.xlsx']

Read parameters for connection to S3 bucket

In [4]:
def read_s3params(filename,section="firevegbucket"):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db


In [5]:
filename = repodir / 'secrets' / 's3info'
s3params = read_s3params(filename)

Start a connection to S3 using the connection parameters: 

In [6]:
s3=boto3.client('s3', s3params['region'],
                        aws_access_key_id=s3params['key'],
                  aws_secret_access_key=s3params['secret'])

Example upload of reports to a folder in a bucket:

In [13]:
inputdir = repodir / "data" / "output-report"
#targetfile = 'fireveg-trait-report-model.xlsx'
targetfile = 'fireveg-trait-records-model.xlsx'
#targetfile = 'fireveg-field-report-model.xlsx'
targetfile = 'fireveg-trait-records-curation.xlsx'

In [14]:
with open(inputdir / targetfile, "rb") as f:
  s3.upload_fileobj(f, 'fireveg-db', 'output-report/' + targetfile)

Upload all original field forms to a input folder

inputdir = repodir / "data" / "field-form"

for fp in inputdir.glob('*.xls[mx]'):
    targetname=os.path.basename(fp)
    if not targetname.startswith('~'):
        print(targetname)
        with open(fp, "rb") as f:
            s3.upload_fileobj(f, 'fireveg-db', 'input-field-form/' + targetname)


